In [348]:
import kaggle 
import pandas as pd
import seaborn as sns
import numpy as np
import re
from sklearn.tree import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

In [406]:
df = pd.read_csv('training_data.csv',encoding = "ISO-8859-1")
dftest = pd.read_csv('test_data.csv',encoding = "ISO-8859-1")

In [407]:
df.head()

,city_name,record_date,AVERAGE_SPEED_DIFF,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,LUMINOSITY,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS,AVERAGE_PRECIPITATION,AVERAGE_RAIN
0,Porto,2019-08-29 07:00:00,Medium,41.5,11.5,71.4,LIGHT,15.0,1019.0,100.0,3.0,NaN,0.0,NaN
1,Porto,2018-08-10 14:00:00,High,41.7,48.3,87.4,LIGHT,21.0,1021.0,53.0,5.0,céu claro,0.0,NaN
2,Porto,2019-09-01 16:00:00,High,38.6,38.4,85.2,LIGHT,26.0,1014.0,61.0,4.0,NaN,0.0,NaN
3,Porto,2019-02-26 11:00:00,High,37.4,61.0,94.1,LIGHT,18.0,1025.0,48.0,4.0,céu claro,0.0,NaN
4,Porto,2019-06-06 12:00:00,Medium,41.6,50.4,77.0,LIGHT,15.0,1008.0,82.0,10.0,NaN,0.0,NaN


In [408]:
df.info()
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6812 entries, 0 to 6811
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city_name                6812 non-null   object 
 1   record_date              6812 non-null   object 
 2   AVERAGE_SPEED_DIFF       6812 non-null   object 
 3   AVERAGE_FREE_FLOW_SPEED  6812 non-null   float64
 4   AVERAGE_TIME_DIFF        6812 non-null   float64
 5   AVERAGE_FREE_FLOW_TIME   6812 non-null   float64
 6   LUMINOSITY               6812 non-null   object 
 7   AVERAGE_TEMPERATURE      6812 non-null   float64
 8   AVERAGE_ATMOSP_PRESSURE  6812 non-null   float64
 9   AVERAGE_HUMIDITY         6812 non-null   float64
 10  AVERAGE_WIND_SPEED       6812 non-null   float64
 11  AVERAGE_CLOUDINESS       4130 non-null   object 
 12  AVERAGE_PRECIPITATION    6812 non-null   float64
 13  AVERAGE_RAIN             563 non-null    object 
dtypes: float64(8), object(6)

In [409]:
df.describe()

,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_PRECIPITATION
count,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.0
mean,40.661010,25.637111,81.143952,16.193482,1017.388139,80.084190,3.058573,0.0
std,4.119023,33.510507,8.294401,5.163492,5.751061,18.238863,2.138421,0.0
min,30.500000,0.000000,46.400000,0.000000,985.000000,14.000000,0.000000,0.0
25%,37.600000,2.275000,75.400000,13.000000,1015.000000,69.750000,1.000000,0.0
50%,40.700000,12.200000,82.400000,16.000000,1017.000000,83.000000,3.000000,0.0
75%,43.500000,36.200000,87.400000,19.000000,1021.000000,93.000000,4.000000,0.0
max,55.900000,296.500000,112.000000,35.000000,1033.000000,100.000000,14.000000,0.0


In [410]:
print(df.isna().sum())

city_name                     0
record_date                   0
AVERAGE_SPEED_DIFF            0
AVERAGE_FREE_FLOW_SPEED       0
AVERAGE_TIME_DIFF             0
AVERAGE_FREE_FLOW_TIME        0
LUMINOSITY                    0
AVERAGE_TEMPERATURE           0
AVERAGE_ATMOSP_PRESSURE       0
AVERAGE_HUMIDITY              0
AVERAGE_WIND_SPEED            0
AVERAGE_CLOUDINESS         2682
AVERAGE_PRECIPITATION         0
AVERAGE_RAIN               6249
dtype: int64


In [411]:
print(df.AVERAGE_SPEED_DIFF.unique())
print(df.LUMINOSITY.unique())
print(df.AVERAGE_CLOUDINESS.unique())
print(df.city_name.unique())
print(df.AVERAGE_RAIN.unique())
print(df.AVERAGE_PRECIPITATION.unique())
print(df.record_date.unique())

['Medium' 'High' 'None' 'Low' 'Very_High']
['LIGHT' 'LOW_LIGHT' 'DARK']
[nan 'céu claro' 'nuvens dispersas' 'céu pouco nublado' 'céu limpo'
 'algumas nuvens' 'nuvens quebrados' 'tempo nublado' 'nuvens quebradas'
 'nublado']
['Porto']
[nan 'chuva fraca' 'chuva' 'chuva leve' 'chuvisco fraco' 'chuva moderada'
 'trovoada com chuva leve' 'aguaceiros' 'aguaceiros fracos'
 'chuva de intensidade pesada' 'trovoada com chuva'
 'chuva de intensidade pesado' 'chuva forte' 'chuvisco e chuva fraca']
[0.]
['2019-08-29 07:00:00' '2018-08-10 14:00:00' '2019-09-01 16:00:00' ...
 '2018-10-02 04:00:00' '2019-01-30 01:00:00' '2019-06-15 21:00:00']


In [412]:
#city_name e AVERAGE_PRECIPITATION só tem uma entrada por isso são inuteis
df_prepared=df.drop(['city_name','AVERAGE_PRECIPITATION'],axis=1)
dftest=dftest.drop(['city_name','AVERAGE_PRECIPITATION'],axis=1)

In [413]:
df_prepared.head()

,record_date,AVERAGE_SPEED_DIFF,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,LUMINOSITY,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS,AVERAGE_RAIN
0,2019-08-29 07:00:00,Medium,41.5,11.5,71.4,LIGHT,15.0,1019.0,100.0,3.0,NaN,NaN
1,2018-08-10 14:00:00,High,41.7,48.3,87.4,LIGHT,21.0,1021.0,53.0,5.0,céu claro,NaN
2,2019-09-01 16:00:00,High,38.6,38.4,85.2,LIGHT,26.0,1014.0,61.0,4.0,NaN,NaN
3,2019-02-26 11:00:00,High,37.4,61.0,94.1,LIGHT,18.0,1025.0,48.0,4.0,céu claro,NaN
4,2019-06-06 12:00:00,Medium,41.6,50.4,77.0,LIGHT,15.0,1008.0,82.0,10.0,NaN,NaN


In [414]:
#transformei o LUMINOSITY podemos dar drop mais tarde
df_prepared['LUMINOSITY'].replace({"DARK": 0, "LOW_LIGHT": 1, "LIGHT": 2}, inplace=True)
dftest['LUMINOSITY'].replace({"DARK": 0, "LOW_LIGHT": 1, "LIGHT": 2}, inplace=True)


In [415]:
#Quantificar AVERAGE_CLOUDINESS
df_prepared['AVERAGE_CLOUDINESS'].replace({"céu limpo": 0, "céu claro": 1, "nuvens dispersas": 2,"nuvens quebrados": 3,"nuvens quebradas": 3,"algumas nuvens": 4,"céu pouco nublado":5,'tempo nublado': 6,'nublado': 6}, inplace=True)
dftest['AVERAGE_CLOUDINESS'].replace({"céu limpo": 0, "céu claro": 1, "nuvens dispersas": 2,"nuvens quebrados": 3,"nuvens quebradas": 3,"algumas nuvens": 4,"céu pouco nublado":5,'tempo nublado': 6,'nublado': 6}, inplace=True)

In [416]:
#Dei drop porque tem varios nulos e podem não ser tão impactantes
#tratamos e testamos mais tarde
df_prepared=df_prepared.drop(['AVERAGE_RAIN'],axis=1)
dftest=dftest.drop(['AVERAGE_RAIN'],axis=1)


In [417]:
#Fill null AVERAGE_CLOUDINESS
df_prepared['AVERAGE_CLOUDINESS']=df_prepared['AVERAGE_CLOUDINESS'].fillna(method='bfill')
df_prepared['AVERAGE_CLOUDINESS']=df_prepared['AVERAGE_CLOUDINESS'].fillna(method='ffill')
dftest['AVERAGE_CLOUDINESS']=df_prepared['AVERAGE_CLOUDINESS'].fillna(method='bfill')
dftest['AVERAGE_CLOUDINESS']=df_prepared['AVERAGE_CLOUDINESS'].fillna(method='ffill')

In [418]:
#Drop luminosity pode não ser importante
df_prepared=df_prepared.drop(['LUMINOSITY'],axis=1)
dftest=dftest.drop(['LUMINOSITY'],axis=1)

In [419]:
#Passar data para apenas a hora e passar a inteiro
df_prepared['record_date']=df_prepared['record_date'].str.slice(11,-6).astype(int)
dftest['record_date']=dftest['record_date'].str.slice(11,-6).astype(int)

In [392]:
#Passar main column para numeros
#df_prepared['AVERAGE_SPEED_DIFF'].replace({"None": 0, "Low": 1, "Medium": 2, "High": 3, "Very_High": 4}, inplace=True)
df_prepared['AVERAGE_SPEED_DIFF'].replace({0:"None", 1:"Low", 2:"Medium", 3:"High", 4:"Very_High"}, inplace=True)
df_prepared.head()

,record_date,AVERAGE_SPEED_DIFF,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS
0,7,Medium,41.5,11.5,71.4,15.0,1019.0,100.0,3.0,céu claro
1,14,High,41.7,48.3,87.4,21.0,1021.0,53.0,5.0,céu claro
2,16,High,38.6,38.4,85.2,26.0,1014.0,61.0,4.0,céu claro
3,11,High,37.4,61.0,94.1,18.0,1025.0,48.0,4.0,céu claro
4,12,Medium,41.6,50.4,77.0,15.0,1008.0,82.0,10.0,nuvens dispersas


In [420]:
df_prepared.head()
dftest.head()

,record_date,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS
0,23,39.2,0.0,91.0,8.0,1026.0,71.0,1.0,1.0
1,20,42.5,12.2,76.8,11.0,1020.0,93.0,4.0,1.0
2,5,45.9,0.0,86.3,14.0,1017.0,93.0,0.0,1.0
3,17,33.2,51.7,89.9,22.0,1016.0,77.0,4.0,1.0
4,6,44.0,3.5,85.5,12.0,1004.0,100.0,9.0,2.0


In [421]:
x = df_prepared.drop(['AVERAGE_SPEED_DIFF'], axis=1)
y = df_prepared['AVERAGE_SPEED_DIFF'].to_frame()

In [422]:
x

,record_date,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_CLOUDINESS
0,7,41.5,11.5,71.4,15.0,1019.0,100.0,3.0,1.0
1,14,41.7,48.3,87.4,21.0,1021.0,53.0,5.0,1.0
2,16,38.6,38.4,85.2,26.0,1014.0,61.0,4.0,1.0
3,11,37.4,61.0,94.1,18.0,1025.0,48.0,4.0,1.0
4,12,41.6,50.4,77.0,15.0,1008.0,82.0,10.0,2.0
...,...,...,...,...,...,...,...,...,...
6807,9,41.2,3.7,86.9,9.0,1030.0,71.0,4.0,1.0
6808,18,38.9,51.9,76.9,17.0,1020.0,63.0,6.0,1.0
6809,4,39.6,0.0,89.1,15.0,1020.0,39.0,4.0,1.0
6810,1,41.6,0.0,85.5,8.0,1018.0,81.0,1.0,4.0


In [423]:
y

,AVERAGE_SPEED_DIFF
0,Medium
1,High
2,High
3,High
4,Medium
...,...
6807,Low
6808,Medium
6809,None
6810,None


In [424]:
x_train = x
y_train = y
x_test = dftest

In [425]:
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)
predictions = clf.predict(x_test)
predictions

array(['None', 'Medium', 'None', ..., 'None', 'Very_High', 'Low'],
      dtype=object)

In [341]:
lm = LinearRegression()
lm.fit(x_train,y_train)
preditcions = lm.predict(x_test)
preditcions

array([[0.69698539],
       [1.17806407],
       [0.42232671],
       ...,
       [0.39679915],
       [3.05597154],
       [1.3077712 ]])

In [136]:
test = pd.read_csv('example_submission.csv',encoding = "ISO-8859-1")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   RowId       1500 non-null   int64 
 1   Speed_Diff  1500 non-null   object
dtypes: int64(1), object(1)
memory usage: 23.6+ KB


In [139]:
test.tail()

,RowId,Speed_Diff
1495,1496,Medium
1496,1497,Medium
1497,1498,Medium
1498,1499,Medium
1499,1500,Medium


In [138]:
predictions.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [426]:
ids = list(range(1,1500))
submission = pd.DataFrame(kmeans.labels_,columns=['Speed_Diff'])
submission.rename(columns={'Unnamed: 0': 'RowId'},inplace=True)
submission.to_csv('submission.csv',index=True)

In [427]:
submission = pd.read_csv('submission.csv',encoding = "ISO-8859-1")
submission['Speed_Diff'].replace({0:"None", 1:"Low", 2:"Medium", 3:"High", 4:"Very_High"}, inplace=True)
submission.rename(columns={'Unnamed: 0': 'RowId'},inplace=True)
submission["RowId"] = 1 + submission["RowId"]
submission.to_csv('submission2.csv',index=False)
submission = pd.read_csv('submission2.csv',encoding = "ISO-8859-1")
submission.head()

,RowId,Speed_Diff
0,1,Low
1,2,Very_High
2,3,Very_High
3,4,None
4,5,Very_High


In [350]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(x_train)
kmeans.labels_

array([2, 0, 0, ..., 3, 2, 0], dtype=int32)

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2,sharey=True,figsize(10,6))
ax1.set_title('k Means')
ax1.scatter(x[])